In [ ]:
# 예제 : https://teddylee777.github.io/langchain/rag-tutorial/
#
# LangChain 설치 및 업데이트
!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub python-dotenv unstructured chromadb faiss-cpu rank_bm25 python-docx sqlalchemy

In [ ]:
import os

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [2]:
# 단계1: 폴더내 모든 문서 로딩
#!pip install unstructured

from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(".", glob="./doc_test/*.txt", show_progress=True)
docs = loader.load()

print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[1].page_content[:200]}")
print(f"\n[metadata]\n{docs[1].metadata}\n")

100%|██████████| 9/9 [00:03<00:00,  2.32it/s]

문서의 수: 9

[페이지내용]
대학교 학자금대출 제도

목적 : 대학생 자녀를 가진 임직원의 학자금을 대출함으로써 임직원의 복지향상, 근로의욕의 제고 및 장기근속 유도

도입기준

대학생 자녀를 가진 정규직 임직원

지급신청 : 부문장(본부는 본부장) 전결로 그룹웨어를 통해 신청

대학교 학자금대출 신청서(그룹웨어)

재학증명서, 입학증명서 중 1종 첨부

지원절차

그룹웨어를 통해 ‘

[metadata]
{'source': 'doc_test/06.대학교 학자금 대출 지원 제도_20.02.01.txt'}



In [12]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

print(f'*chunk 수:{len(splits)}\n')
print(f'*splits[0]:\n{splits[0]}')


*chunk 수:40

*splits[0]:
page_content='경조사 지원 규정\n\n1. 적용대상\n\n계약직 사원을 포함한 임직원(고문 및 용역은 사장이 별도로 결정)\n\n2. 경조사 지원기준\n\n구 분\n\n내 역\n\n휴가(일)\n\n금액(원)\n\n기타\n\n경 사\n\n본인 결혼\n\n자녀 결혼\n\n형제자매결혼\n\n자녀 출산(배우자)\n\n부모 회갑\n\n배우자 부모회갑\n\n부모고희(칠순)\n\n배우자부모고희\n\n5\n\n1\n\n1\n\n10\n\n1\n\n1\n\n1\n\n1\n\n500,000\n\n300,000\n\n100,000\n\n100,000\n\n200,000\n\n200,000\n\n300,000\n\n300,000\n\n화환 지급\n\n화환 지급\n\n1회 분할 사용 가능\n\n조 사\n\n본인 사망\n\n배우자 사망\n\n부모 사망\n\n자녀 사망\n\n배우자 부모 사망\n\n형제자매 사망\n\n조부모/외조부모 사망\n\n5\n\n5\n\n5\n\n5\n\n3\n\n3\n\n1,000,000\n\n1,000,000\n\n1,000,000\n\n1,000,000 500,000\n\n300,000\n\n200,000\n\n조화 지급\n\n조화 지급\n\n조화 지급\n\n조화 지급\n\n조화 지급\n\n조화 지급' metadata={'source': 'doc_test/03.경조사지원규정_12.01.01.txt'}


In [18]:
splits[1].page_content

'조화 지급\n\n조화 지급\n\n조화 지급\n\n조화 지급\n\n조화 지급\n\n조화 지급\n\n3. 신청서류\n\n경조금 : 경조금지급신청서 및 증빙서류 1부\n\n휴 가: 휴가원 1부\n\n4. 경조휴가일수\n\n공휴일과 경조사가 중복되었을 경우 휴가일수는 공휴일을 제외하여 계산한다.\n\n5. 결혼퇴직\n\n결혼퇴직의 경우 퇴직 1개월 이내에 결혼할 시에는 위의 기준에 의거하여 지급한다.\n\n6. 기타\n\n경조금 신청 시 휴가 신청도 같이 진행해야 함이 원칙임\n\n특별한 경우 사업부장 합의 시 경조금 신청일 이후 신청 가능(6개월 이내)\n\n분할 사용 불가, 발생일 이전 신청 불가.\n\n부 칙\n\n(시행일) 이 규정은 2007년 9월 14일부터 시행한다.\n\n(개정일) 이 규정은 2012년 1월 1일부터 개정 시행한다.\n\n경조사지원규정 1/1'

In [23]:
# character_text_splitter 와 RecursiveCharacterTextSplitter 비교
# => 같은 문서를 불러와서 100개로 split 할때 출력을 보여줌. RecursiveCharacterTextSplitter가 더 좋음.
# => RecursiveCharacterTextSplitter가 의미적으로 가장 연관성이 강한 텍스트 조각인 것처럼 보이는 
# 모든 단락(그리고 문장, 단어)을 가능한 한 길게 유지하려는 효과가 있다.
with open("./doc_test/03.경조사지원규정_12.01.01.txt", "r") as f:
    text = f.read()[:500]
    
character_text_splitter = CharacterTextSplitter(
    chunk_size=100, chunk_overlap=10, separator=" "
)
for sent in character_text_splitter.split_text(text):
    print(sent)
    print("--"*10)
    

print("===" * 20)

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=10
)
for sent in recursive_text_splitter.split_text(text):
    print(sent)
    print("--"*10)

Created a chunk of size 105, which is longer than the specified 100


경조사 지원 규정
1. 적용대상
계약직 사원을 포함한 임직원(고문 및 용역은 사장이 별도로 결정)
2. 경조사 지원기준
구 분
내 역
휴가(일)
금액(원)
기타
경 사
본인
--------------------
사
본인 결혼
자녀 결혼
형제자매결혼
자녀 출산(배우자)
부모 회갑
배우자
--------------------
부모회갑
부모고희(칠순)
배우자부모고희
5
1
1
10
1
1
1
1
500,000
300,000
100,000
100,000
200,000
200,000
300,000
300,000
화환
--------------------
지급
화환 지급
1회 분할 사용 가능
조 사
본인 사망
배우자 사망
부모 사망
자녀 사망
배우자 부모 사망
형제자매 사망
조부모/외조부모
--------------------
사망
-
5
5
5
5
3
3
1,000,000
1,000,000
1,000,000
1,000,000 500,000
300,000
200,000
조화 지급
조화 지급
조화
--------------------
지급
조화 지급
조화 지급
조화 지급
조화 지급
3. 신청서류
경조금 : 경조금지급신청서 및 증빙서류 1부
휴 가: 휴가원 1부
4. 경조휴가일수
공휴일과 경조사가 중
--------------------
경조사 지원 규정
1. 적용대상
계약직 사원을 포함한 임직원(고문 및 용역은 사장이 별도로 결정)
2. 경조사 지원기준
구 분
내 역
휴가(일)
금액(원)
기타
경 사
본인 결혼
--------------------
경 사
본인 결혼
자녀 결혼
형제자매결혼
자녀 출산(배우자)
부모 회갑
배우자 부모회갑
부모고희(칠순)
배우자부모고희
5
1
1
10
1
1
1
1
500,000
300,000
--------------------
300,000
100,000
100,000
200,000
200,000
300,000
300,000
화환 지급
화환 지급
1회 분할 사용 가능
조 사
본인 사망
배우자 사망
--------------------
배우자

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

# openai 임베딩 모델 이용
#semantic_text_splitter = SemanticChunker(
#    OpenAIEmbeddings, add_start_index=True)

# bge 임베딩 모델 이용 => bge는 영문에 특화되어 있어서 영문문서일때는 좋음.
#semantic_text_splitter = SemanticChunker(
#    HuggingFaceBgeEmbeddings, add_start_index=True)

# hugggingface에 다른 모델 이용
#model_name = "BAAI/bge-small-en"
model_name = "bongsoo/kpf-sbert-v1"
#model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
custom_embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    #model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# SemanticChunker 를 생성합니다.
semantic_text_splitter = SemanticChunker(
    custom_embedding_model, add_start_index=True)

# chain of density 논문의 일부 내용을 불러옵니다
with open("./doc_test/25.출장여비규정_16.04.01.txt", "r") as f:
    text = f.read()

for sent in semantic_text_splitter.split_text(text):
    print(sent)
    print("===" * 20)

In [ ]:
# sqlalchemy 에러가 나면 업그레이드 해준다.
#!pip install --upgrade sqlalchemy

In [34]:
# 단계 3: 임베딩
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS, Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

# 단계 3, 4: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
#vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

#vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

# 단계 5: 리트리버 생성(Create Retriever)
# 사용자의 질문(query) 에 부합하는 문서를 검색합니다.
# 유사도 높은 K 개의 문서를 검색합니다.
k = 3

# BM25 retriver 생성
# (Sparse) bm25 retriever and (Dense) faiss retriever 를 초기화 합니다.
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = k

# 임베딩 retiver 모델을 불러옴.
model_name = "bongsoo/kpf-sbert-v1"
#model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
custom_embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    #model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 임베딩 retriver 생성
#faiss_vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))
faiss_vectorstore = FAISS.from_documents(documents=splits, embedding=custom_embedding_model)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": k})

# bm25 + 임베딩 retriver 합쳐서 만듬.
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)


In [36]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

print(f'RAG-prompt:\n{prompt}')

# 단계 7: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


RAG-prompt:
input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [37]:
# 단계 8: 체인 생성(Create Chain)
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "1박2일 서울 출장시 식비는 얼마인가요?"
response = rag_chain.invoke(question)

# 결과 출력
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

문서의 수: 9
[HUMAN]
1박2일 서울 출장시 식비는 얼마인가요?

[AI]
서울 출장시 식비는 1일당 20,000원입니다. 식비는 실비로 정산 가능하며, 상한액은 서울특별시 70,000원입니다.


In [ ]:
# HuggingFaceHub 객체 생성
# 모델들은 아래 사이트 참조
# https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard
from langchain.llms import HuggingFaceHub

#repo_id = "google/flan-t5-xxl"
repo_id = "hwkwon/S-SOLAR-10.7B-v1.5"

llm_model = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.1, "max_length": 512}
)

In [ ]:
llm_model.invoke("한국의 수도는 어디인가요?")

In [ ]:
# https://teddylee777.github.io/langchain/langchain-tutorial-02/

In [ ]:
#모델을 직접 다운로드 후 로컬(local)에서 추론Permalink
#이전 방식은 허깅페이스 서버에서 선택된 모델로 추론하고, 이에 대한 답변을 반환받는 방식입니다.
#추론 방식이 간편하지만, 서버의 성능에 따라 다르지만 추론 속도가 대체적으로 오래 걸리는 편입니다. 
#따라서, 결과를 받는데 시간이 오래 걸리거나, 혹은 답변의 지연시간이 긴 경우, Timeout 에러가 발생할 수 있습니다
#만약, 좋은 성능의 GPU 를 탑재한 서버가 있다면, 로컬에 모델을 직접 다운로드 받아 GPU 부스트를 받아서 추론할 수 있습니다. 아래는 예시코드 입니다.

import os
# 허깅페이스 모델/토크나이저를 다운로드 받을 경로
# (예시)
# os.environ['HF_HOME'] = '/home/jovyan/work/tmp'
os.environ['HF_HOME'] = './model'

In [ ]:
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline

# HuggingFace Model ID
model_id = 'beomi/llama-2-ko-7b'
#gwonny/nox-solar-10.7b-v4-kolon-all-5-v2.0 

# HuggingFacePipeline 객체 생성
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id, 
    device=0,               # -1: CPU(default), 0번 부터는 CUDA 디바이스 번호 지정시 GPU 사용하여 추론
    task="text-generation", # 텍스트 생성
    model_kwargs={"temperature": 0.1, 
                  "max_length": 512},
)

# 템플릿
template = """질문: {question}

답변: """

# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(template)

# LLM Chain 객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)